#### Project size estimation

In [8]:
import re

def remove_comments_and_empty_lines_PY(code):
    # Remove single-line comments
    code = re.sub(r'#.*', '', code)
    # Remove multi-line comments
    code = re.sub(r'\'\'\'(.*?)\'\'\'|\"\"\"(.*?)\"\"\"', '', code, flags=re.DOTALL)
    # Remove empty lines
    code = re.sub(r'\n\s*\n', '\n', code)
    # Strip leading/trailing whitespace from each line
    code = '\n'.join(line.strip() for line in code.splitlines() if line.strip())
    return code
def remove_comments_and_empty_lines_JS(code):
    # Remove single-line comments
    code = re.sub(r'//.*', '', code)
    # Remove multi-line comments
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
    # Remove empty lines
    code = re.sub(r'\n\s*\n', '\n', code)
    # Strip leading/trailing whitespace from each line
    code = '\n'.join(line.strip() for line in code.splitlines() if line.strip())
    return code
def count_pure_code_lines(code, lang):
    if lang == 'py':
        return len(remove_comments_and_empty_lines_PY(code).splitlines())
    if lang == 'js':
        return len(remove_comments_and_empty_lines_JS(code).splitlines())

In [16]:
code_files = [
    'web/app/admin.py',
    'web/app/apps.py',
    'web/app/consumers.py',
    'web/app/forms.py',
    'web/app/models.py',
    'web/app/routing.py',
    'web/app/urls.py',
    'web/app/views.py',
    'web/app/assistant/emotional_journal.py',
    'web/app/assistant/helpers.py',
    'web/app/assistant/moderation.py',
    'web/app/assistant/responder.py',
    'web/app/assistant/settings.py',
    'web/app/assistant/tools.py',
    'web/app/assistant/user_profile.py',
    'web/app/assistant/Tools/tools_functions.py'
    ]

code_lines = 0
for code_file in code_files:
    with open(code_file, 'r') as file:
        code_lines += count_pure_code_lines(file.read(), 'py')

code_lines_js = 0   
with open('web/app/templates/app/chat.html', 'r') as file:
    code_lines_js += count_pure_code_lines(file.read(), 'js') - 247
    
print(f"py: {code_lines}\njs: {code_lines_js}\ntotal: {code_lines + code_lines_js}")

py: 1264
js: 586
total: 1850


#### Recommendations utils

In [13]:
from web.app.assistant.recommender import *
from dotenv import load_dotenv
import os, openai, textwrap

load_dotenv()
openai.api_key = os.getenv("OPENAI_KEY")

recommender = Recommender(
    gpt_model="gpt-4o-mini",
    recommendations_tree_path="web/app/assistant/Recommendations/recomendation_tree.pkl",
    recommendations_path="web/app/assistant/Recommendations/recommendations.txt"
    )

In [ ]:
categories = [
    "Sport", "Nutrition","Sleep", 
    "Stress", "Mental Health", "Social", 
    "Self-Care", "Motivation", "Resilience", 
    "Body", "Mindfulness", "Positive Thinking"
]
new_rec, usage = await recommender.generate_recommendation(categories)
print(textwrap.fill(new_rec, width=50), usage)

In [ ]:
vector, usage = await recommender.generate_categories_v(categories, new_rec)
for i in list(zip(categories, vector)):
    print(f"{i[0]}: {i[1]}")
print(usage)

Incorporate a daily routine of moderate exercise,
balanced nutrition, and mindfulness practices to
enhance your overall well-being.
Sport: 7
Nutrition: 8
Sleep: 0
Stress: 0
Mental Health: 0
Social: 0
Self-Care: 0
Motivation: 0
Resilience: 0
Body: 0
Mindfulness: 8
Positive Thinking: 6


In [ ]:
recommender.save_recomendation(new_rec, vector)

In [4]:
recommender.build_and_save_rec_tree()

Median: v: [1, 2, 3] i:1
Distance  to [2, 3, 5]:  2.449489742783178
v: [1, 3, 4, 5, 1, 6] i:0
v: [1, 2, 3] i:1 <-----
Depth:  0 
 __________
Median: v: [1, 3, 4, 5, 1, 6] i:0
Distance  to [2, 3, 5]:  1.4142135623730951
v: [1, 3, 4, 5, 1, 6] i:0 <-----
Depth:  1 
 __________


In [ ]:
vector = []
rec = recommender.get_recomendations(vector, n_max=10)